In [ ]:
%run ./model_end_events_for_outages_METHODS.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
import string
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy
from functools import reduce

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
from EEMSP import EEMSP
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from MECPOCollection import MECPOCollection
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer
from CustomJSON import CustomEncoder, CustomWriter

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GroupShuffleSplit, GridSearchCV, RandomizedSearchCV, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn import preprocessing

import tensorflow as tf
from tensorflow import keras

import scipy

from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

# ---------------------------------------------------------------------------------------------------

In [ ]:
fig_num=0

# save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models_00_05'
save_dir_model_base = r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230615\Models'

merged_df_full=pd.read_pickle(os.path.join(save_dir_model_base, 'merged_df_full.pkl'))

# =========================================================
# =========================================================

# EEMSP

In [ ]:
merged_df_full = merged_df_full[:10000]

In [ ]:
trsf_pole_nbs = list(set(
    merged_df_full.index.get_level_values(1).unique().tolist()
))
#-------------------------
conn_eemsp = Utilities.get_eemsp_oracle_connection()
df_eemsp_OG = build_df_eemsp(conn_eemsp, trsf_pole_nbs, batch_size=1000, verbose=True, n_update=10)
#-------------------------
df_eemsp_OG = Utilities_df.make_all_column_names_lowercase(df_eemsp_OG)

In [ ]:
# conn_dev = Utilities.get_athena_dev_aws_connection()
conn_dev = Utilities.get_athena_prod_aws_connection()
#-----
sql_NEW = """
SELECT * 
FROM meter_events.eems_transformer_nameplate
WHERE location_nb IN ({})
""".format(Utilities_sql.join_list(trsf_pole_nbs, quotes_needed=True))
#-----
df_eemsp_NEW = pd.read_sql_query(sql_NEW, conn_dev)
df_eemsp_NEW = df_eemsp_NEW[df_eemsp_OG.columns]
#-----
df_eemsp_NEW['install_dt'] = pd.to_datetime(df_eemsp_NEW['install_dt'])
df_eemsp_NEW['removal_dt'] = pd.to_datetime(df_eemsp_NEW['removal_dt'])

In [ ]:
gp_by = ['eqseq_id', 'equip_id', 'serial_nb', 'transaction_id', 'install_dt']
df_eemsp_OG =  df_eemsp_OG.sort_values(by=gp_by, ignore_index=True)
df_eemsp_NEW = df_eemsp_NEW.sort_values(by=gp_by, ignore_index=True)

In [ ]:
df_eemsp_NEW.groupby(gp_by).ngroups

In [ ]:
df_eemsp_OG.groupby(gp_by).ngroups

In [ ]:
len(set(list(df_eemsp_OG.groupby(gp_by).groups.keys())).difference(
    set(list(df_eemsp_NEW.groupby(gp_by).groups.keys()))
))

In [ ]:
len(set(list(df_eemsp_NEW.groupby(gp_by).groups.keys())).difference(
    set(list(df_eemsp_OG.groupby(gp_by).groups.keys()))
))

In [ ]:
gps_not_in_NEW = list(set(list(df_eemsp_OG.groupby(gp_by).groups.keys())).difference(
    set(list(df_eemsp_NEW.groupby(gp_by).groups.keys()))
))
gps_overlap = list(set(list(df_eemsp_OG.groupby(gp_by).groups.keys())).intersection(
    set(list(df_eemsp_NEW.groupby(gp_by).groups.keys()))
))
#-----
df_eemsp_OG_gpd  = df_eemsp_OG.groupby(gp_by)
df_eemsp_NEW_gpd = df_eemsp_NEW.groupby(gp_by)

In [ ]:
if len(gps_not_in_NEW)>0:
    df_not_in_NEW = pd.concat([df_eemsp_OG_gpd.get_group(x) for x in gps_not_in_NEW])
else:
    df_not_in_NEW = None
df_eemsp_OG_overlap  = pd.concat([df_eemsp_OG_gpd.get_group(x) for x in gps_overlap])
df_eemsp_NEW_overlap = pd.concat([df_eemsp_NEW_gpd.get_group(x) for x in gps_overlap])

In [ ]:
df_eemsp_OG_overlap.equals(df_eemsp_NEW_overlap)

In [ ]:
df_eemsp_OG_overlap  = df_eemsp_OG_overlap.sort_values(by=gp_by, ignore_index=True)
df_eemsp_NEW_overlap = df_eemsp_NEW_overlap.sort_values(by=gp_by, ignore_index=True)

In [ ]:
df_eemsp_OG_overlap.equals(df_eemsp_NEW_overlap)

In [ ]:
diffs = Utilities_df.get_dfs_diff(df_eemsp_OG_overlap, df_eemsp_NEW_overlap)

In [ ]:
len(diffs)

In [ ]:
diffs

# =========================================================
# =========================================================

# DOVS

In [ ]:
# outg_rec_nbs = merged_df_full.index.get_level_values(0).unique().tolist()
# conn_dovs = Utilities.get_utldb01p_oracle_connection()
# sql_OG = DOVSOutages_SQL.build_sql_std_outage(outg_rec_nbs=outg_rec_nbs)
# df_DOVS_OG = pd.read_sql_query(sql_OG.get_sql_statement(), conn_dovs)
# #-----
# df_DOVS_OG = Utilities_df.make_all_column_names_lowercase(df_DOVS_OG)

In [ ]:
gp_by = ['outg_rec_nb', 'location_id', 'premise_nb', 'amed_fl']

In [ ]:
outg_rec_nbs = merged_df_full.index.get_level_values(0).unique().tolist()
conn_dovs = Utilities.get_utldb01p_oracle_connection()
sql_OG = """
SELECT *
FROM DOVSADM.DOVS_OUTAGE_FACT DOV
    LEFT OUTER JOIN DOVSADM.DOVS_MASTER_GEO_DIM DOV1 ON DOV.OPERATING_UNIT_ID=DOV1.OPRTG_UNT_ID AND DOV.STATE_ABBR_TX=DOV1.STATE_ID AND DOV.OPCO_NBR=DOV1.OPCO_ID AND DOV.DISTRICT_NB=DOV1.DISTRICT_ID AND DOV.SRVC_CNTR_NB=DOV1.AREA_ID AND DOV.GIS_CRCT_NB=DOV1.GIS_CIRCUIT_ID
    LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_ATTRIBUTES_DIM DOV2 ON DOV.OUTG_REC_NB=DOV2.OUTG_REC_NB
    LEFT OUTER JOIN DOVSADM.DOVS_CLEARING_DEVICE_DIM DOV3 ON DOV.DEVICE_CD=DOV3.DEVICE_CD
    LEFT OUTER JOIN DOVSADM.DOVS_EQUIPMENT_TYPES_DIM DOV4 ON DOV.EQUIPMENT_CD=DOV4.EQUIPMENT_CD
    LEFT OUTER JOIN DOVSADM.DOVS_OUTAGE_CAUSE_TYPES_DIM DOV5 ON DOV.MJR_CAUSE_CD=DOV5.MJR_CAUSE_CD AND DOV.MNR_CAUSE_CD=DOV5.MNR_CAUSE_CD
    LEFT OUTER JOIN DOVSADM.DOVS_PREMISE_DIM PRIM ON DOV.OUTG_REC_NB=PRIM.OUTG_REC_NB
WHERE DOV.OUTG_REC_NB IN ({})
AND   DOV.MJR_CAUSE_CD <> 'NI'
AND   DOV.DEVICE_CD <> 85
AND   DOV2.INTRPTN_TYP_CD = 'S'
AND   DOV2.CURR_REC_STAT_CD = 'A'
""".format(Utilities_sql.join_list(outg_rec_nbs, quotes_needed=False))
df_DOVS_OG = pd.read_sql_query(sql_OG, conn_dovs)
#-----
df_DOVS_OG = Utilities_df.make_all_column_names_lowercase(df_DOVS_OG)

In [ ]:
df_DOVS_OG

In [ ]:
degen_cols = df_DOVS_OG.columns.value_counts()[df_DOVS_OG.columns.value_counts()>1].index.tolist()
degen_cols

In [ ]:
truly_degen_cols = []
for col in degen_cols:
    if not all(df_DOVS_OG[col].nunique(axis=1)<=1):
        truly_degen_cols.append(col)

In [ ]:
truly_degen_cols

In [ ]:
# Get rid of degen cols for now by keeping first
df_DOVS_OG=df_DOVS_OG.groupby(df_DOVS_OG.columns, axis=1).first()

In [ ]:
# NOTE: Apparently outg_rec_nb is type BIGINT in new DOVS table
# conn_dev = Utilities.get_athena_dev_aws_connection()
conn_dev = Utilities.get_athena_prod_aws_connection()
#-----
sql_NEW = """
SELECT * 
FROM meter_events.dovs_outage_fact_premise
WHERE outg_rec_nb IN ({})
AND   mjr_cause_cd <> 'NI'
AND   device_cd <> 85
AND   intrptn_typ_cd = 'S'
AND   curr_rec_stat_cd = 'A'
""".format(Utilities_sql.join_list(outg_rec_nbs, quotes_needed=False))
#-----
df_DOVS_NEW = pd.read_sql_query(sql_NEW, conn_dev)


In [ ]:
set(df_DOVS_NEW.columns).intersection(truly_degen_cols)

In [ ]:
df_DOVS_NEW = df_DOVS_NEW[list(set(df_DOVS_NEW.columns).intersection(set(df_DOVS_OG.columns)))]
df_DOVS_OG  = df_DOVS_OG[list(set(df_DOVS_NEW.columns).intersection(set(df_DOVS_OG.columns)))]

In [ ]:
df_DOVS_NEW = df_DOVS_NEW.sort_values(by=gp_by, ignore_index=True)
df_DOVS_OG  = df_DOVS_OG.sort_values(by=gp_by, ignore_index=True)

In [ ]:
df_DOVS_OG.equals(df_DOVS_NEW)

In [ ]:
diffs = Utilities_df.get_dfs_diff(df_DOVS_OG, df_DOVS_NEW)
diffs

In [ ]:
diffs = Utilities_df.get_dfs_diff_approx_ok(df_DOVS_OG, df_DOVS_NEW)
diffs['diffs_df']

In [ ]:
all_na_cols = df_DOVS_NEW.isna().sum()[df_DOVS_NEW.isna().sum()==df_DOVS_NEW.shape[0]].index.tolist()

In [ ]:
df_DOVS_OG  = df_DOVS_OG.drop(columns=all_na_cols)
df_DOVS_NEW = df_DOVS_NEW.drop(columns=all_na_cols)

In [ ]:
df_DOVS_OG.equals(df_DOVS_NEW)

In [ ]:
diffs = Utilities_df.get_dfs_diff(df_DOVS_OG, df_DOVS_NEW)
diffs

In [ ]:
diffs = Utilities_df.get_dfs_diff_approx_ok(df_DOVS_OG, df_DOVS_NEW)
diffs['diffs_df']